# Notebook 1 - Aprendizagem Supervisionada

In [1]:
# Instalação das dependências usadas no notebook (para execução local).
print("Hello World!")

%pip install --upgrade pip
%pip install numpy
%pip install matplotlib
%pip install pandas
%pip install mlxtend


Hello World!
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Importar o Dataset e preparar os dados

### O que será carregado
- `df_male_players`: base de jogadores masculinos (CSV EA FC)
- `df_female_players`: base de jogadores femininos (CSV EA FC)
- As colunas são mantidas como no dataset original; o género é adicionado manualmente na próxima etapa.

In [2]:
import pandas as pd

# Carrega os datasets brutos com jogadores masculinos e femininos.
df_male_players = pd.read_csv("Data/EA_FC/male_players.csv", low_memory=False)
df_female_players = pd.read_csv("Data/EA_FC/female_players.csv")


In [3]:
# Adiciona a coluna de género e junta todos os jogadores num único DataFrame.
df_male_players["gender"] = "M"
df_female_players["gender"] = "F"

df_players_all = pd.concat(
    [df_male_players, df_female_players],
    ignore_index=True
)

# Filtra apenas os jogadores da edição FC 24.
df_players_fifa24 = df_players_all[df_players_all["fifa_version"] == 24].copy()


### Estrutura do dataset unificado
- `df_players_all`: concatenação de todos os jogadores com a coluna `gender` preenchida.
- `df_players_fifa24`: filtro para a versão `fifa_version == 24`, criando uma cópia para evitar avisos de `SettingWithCopy`.
- A filtragem garante que apenas a edição FC 24 entra no treino e avaliação.

# Selecionar os dados relevantes

### Atribui os grupos de posições baseado na principal posição do jogador. Junta numa coluna nova 'position_group'.


In [4]:
def map_position_group(positions_str: str) -> str:
    """
    Mapeia a posição principal do jogador para um grupo simplificado
    (GK, DEF, MID, ATT ou OTHER).
    """
    if pd.isna(positions_str):
        return "OTHER"
    main_pos = positions_str.split(",")[0].strip()
    defenders = {"CB", "LB", "RB", "LWB", "RWB"}
    mids = {"CDM", "CM", "CAM", "LM", "RM"}
    attackers = {"ST", "CF", "LW", "RW"}

    if main_pos == "GK":
        return "GK"
    elif main_pos in defenders:
        return "DEF"
    elif main_pos in mids:
        return "MID"
    elif main_pos in attackers:
        return "ATT"
    else:
        return "OTHER"

# Cria a coluna categórica com os grupos de posição.
df_players_fifa24["position_group"] = df_players_fifa24["player_positions"].apply(map_position_group)


In [5]:
# Seleciona as colunas de atributos que descrevem o jogador.
feature_cols = [
    "pace", "shooting", "passing", "dribbling",
    "defending", "physic",
    "height_cm", "weight_kg", "age",
    "movement_acceleration", "movement_sprint_speed",
    "movement_agility", "movement_balance",
    "power_strength", "power_stamina"
]

# Limpa linhas com valores em falta e separa features (X) e alvo (y).
df_training_data = df_players_fifa24.dropna(subset=feature_cols + ["position_group"]).copy()

feature_matrix = df_training_data[feature_cols]
position_labels = df_training_data["position_group"]


### Conjunto de features e alvo
- `feature_cols`: atributos físicos/técnicos usados como preditores.
- `df_training_data`: registros sem valores em falta nas features e na coluna `position_group`.
- `feature_matrix` (`X`) e `position_labels` (`y`) são extraídos aqui e usados em todo o fluxo.

## Dividir os dados em treino e teste, e escalar as features para melhorar a performance dos modelos

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Divide os dados mantendo a proporção das classes e normaliza as features numéricas.
X_train, X_test, y_train, y_test = train_test_split(
    feature_matrix,
    position_labels,
    test_size=0.2,
    random_state=7213,
    stratify=position_labels
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


### Divisão e normalização
- `train_test_split` estratificado (80/20) para manter a proporção das classes.
- `StandardScaler` ajustado no treino (`X_train`) e aplicado no teste (`X_test`) para evitar vazamento de dados.
- As versões escaladas (`X_train_scaled`, `X_test_scaled`) são usadas por todos os modelos seguintes.

# Treinar e avaliar vários modelos de classificação

Foram utilizados três modelos de classificação: Regressão Logística, Random Forest e K-Nearest Neighbors (KNN). Cada modelo foi treinado com os dados de treino escalados e avaliado com os dados de teste escalados. As métricas de avaliação incluem precisão, recall e F1-score para cada classe

### Avaliação inicial dos modelos
- Modelos comparados: Regressão Logística, Random Forest e KNN.
- Cada `classification_report` apresenta precisão, recall, F1 por classe e médias macro/micro.
- Estes resultados servem como base antes de tunar os hiperparâmetros.

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

# Modelos de base para comparar desempenho.
models = {
    "LogisticRegression": LogisticRegression(max_iter=100000),
    "RandomForest": RandomForestClassifier(random_state=6452),
    "KNN": KNeighborsClassifier()
}

# Treina cada classificador e imprime as métricas para comparação rápida.
for name, clf in models.items():
    clf.fit(X_train_scaled, y_train)
    y_pred = clf.predict(X_test_scaled)

    print("====", name, "====")
    print(classification_report(y_test, y_pred))


==== LogisticRegression ====
              precision    recall  f1-score   support

         ATT       0.84      0.77      0.80       772
         DEF       0.87      0.88      0.87      1353
         MID       0.78      0.80      0.79      1461

    accuracy                           0.82      3586
   macro avg       0.83      0.82      0.82      3586
weighted avg       0.82      0.82      0.82      3586

==== RandomForest ====
              precision    recall  f1-score   support

         ATT       0.85      0.75      0.80       772
         DEF       0.88      0.89      0.88      1353
         MID       0.78      0.82      0.80      1461

    accuracy                           0.83      3586
   macro avg       0.84      0.82      0.83      3586
weighted avg       0.83      0.83      0.83      3586

==== KNN ====
              precision    recall  f1-score   support

         ATT       0.78      0.72      0.75       772
         DEF       0.85      0.86      0.85      1353
         

# Determinar o melhor modelo e otimizar seus hiperparâmetros com GridSearchCV

 Com base nos resultados iniciais, o melhor modelo será selecionado e seus hiperparâmetros serão otimizados usando GridSearchCV para tentar melhorar ainda mais o desempenho.

### Otimização com GridSearchCV
- O melhor modelo (pelo F1 macro) é reavaliado com grelhas de hiperparâmetros específicas.
- `GridSearchCV` usa validação cruzada (`cv=5`) e paraleliza com `n_jobs=-1`.
- O relatório final compara o F1 macro original vs. o otimizado para verificar o ganho real.

In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

# Primeiro, avaliar e selecionar o melhor modelo inicial com F1 macro.
model_scores = {}
for name, clf in models.items():
    y_pred = clf.predict(X_test_scaled)
    score = f1_score(y_test, y_pred, average="macro")
    model_scores[name] = score
    print(f"F1-Score (macro) para {name}: {score:.4f}")

# Escolhe o modelo de maior F1 macro como candidato à afinação.
best_model_name = max(model_scores, key=model_scores.get)
print(f"\nMelhor modelo inicial: {best_model_name} com F1-Score de {model_scores[best_model_name]:.4f}")

# Define grelhas específicas para cada algoritmo antes de rodar o GridSearch.
param_grids = {
    "LogisticRegression": {
        "C": [0.1, 1.0, 10.0],
        "solver": ["liblinear", "saga"]
    },
    "RandomForest": {
        "n_estimators": [100, 200],
        "max_depth": [10, 20, None],
        "min_samples_split": [2, 5],
    },
    "KNN": {
        "n_neighbors": [3, 5, 7],
        "weights": ["uniform", "distance"],
        "metric": ["euclidean", "manhattan"]
    }
}

# Tem de se otimizar o melhor modelo com GridSearchCV usando validação cruzada.
best_model_base = models[best_model_name]
grid_to_use = param_grids[best_model_name]

print(f"\nIniciando GridSearchCV para o modelo: {best_model_name}...")
grid = GridSearchCV(
    estimator=best_model_base,
    param_grid=grid_to_use,
    scoring="f1_macro",
    n_jobs=-1,
    cv=5
)

# Ajusta o modelo com validação cruzada para encontrar a melhor combinação de parâmetros.
grid.fit(X_train_scaled, y_train)

# Avalia o modelo ajustado no hold-out e compara com o baseline.
print("\nMelhores parâmetros encontrados:", grid.best_params_)
best_model_tuned = grid.best_estimator_
y_pred_tuned = best_model_tuned.predict(X_test_scaled)

print("\nRelatório de classificação para o modelo otimizado:")
print(classification_report(y_test, y_pred_tuned))

print(f"F1-Score  do modelo original: {f1_score(y_test, models[best_model_name].predict(X_test_scaled), average='macro'):.4f}")
print(f"F1-Score (macro) do modelo otimizado: {f1_score(y_test, y_pred_tuned, average='macro'):.4f}")


F1-Score (macro) para LogisticRegression: 0.8221
F1-Score (macro) para RandomForest: 0.8264
F1-Score (macro) para KNN: 0.7832

Melhor modelo inicial: RandomForest com F1-Score de 0.8264

Iniciando GridSearchCV para o modelo: RandomForest...

Melhores parâmetros encontrados: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}

Relatório de classificação para o modelo otimizado:
              precision    recall  f1-score   support

         ATT       0.85      0.75      0.80       772
         DEF       0.88      0.89      0.88      1353
         MID       0.78      0.82      0.80      1461

    accuracy                           0.83      3586
   macro avg       0.84      0.82      0.83      3586
weighted avg       0.83      0.83      0.83      3586

F1-Score  do modelo original: 0.8264
F1-Score (macro) do modelo otimizado: 0.8264
